In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/mouth_dataset_split.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')


Done


In [3]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Shared.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [12]:
import numpy as np
import argparse
import dlib
import cv2
import os
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/Shared/shape_predictor_68_face_landmarks.dat')


In [6]:
def get_data(direc):
    X =[]
    y = []
    data = []
    IMG_SIZE = 50
    categories = os.listdir(direc)
    for category in categories:
        path_link = os.path.join(direc, category)
        for image in os.listdir(path_link):
            image_array = cv2.imread(os.path.join(path_link, image),cv2.IMREAD_GRAYSCALE)
            image_array = cv2.resize(image_array, (IMG_SIZE, IMG_SIZE))
            X.append(image_array)
            y.append(int(category))
    return X,y



In [7]:
X_train,y_train = get_data("/content/mouth_dataset_split/train")
X_test,y_test = get_data("/content/mouth_dataset_split/test")


In [8]:
X_train = np.array(X_train).reshape(-1,50,50,1)
X_test = np.array(X_test).reshape(-1,50,50,1)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)
y_train = y_train.flatten()
y_test = y_test.flatten()

In [9]:
train_dir = "/content/mouth_dataset_split/train"
test_dir = "/content/mouth_dataset_split/test"

In [10]:
batch_train = 2048
batch_val = 512

In [13]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=2,
                                   zoom_range=0.2,
                                   validation_split=0.2, 
                                   dtype=np.float32)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Source directory
    target_size=(50, 50),  #Resized to this size
    batch_size=batch_train,
    color_mode='grayscale',
    subset='training',
    class_mode='binary')

Found 3234 images belonging to 2 classes.


In [15]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,  # Source directory
    target_size=(50, 50),  # Resized
    batch_size=batch_val,
    color_mode='grayscale',
    subset='validation',
    class_mode='binary')

Found 807 images belonging to 2 classes.


In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks = [
    EarlyStopping(monitor = 'val_loss',
                          min_delta = 0, #minimium amount of change to count as an improvement
                          patience = 10, # how many epochs to wait before stopping
                          verbose = 1,
                          restore_best_weights = True),
    ReduceLROnPlateau(factor=0.9, patience=10, min_lr=0.001),
    ModelCheckpoint('EnsembleMouthModel.h5', verbose=1, save_best_only=True)#, save_weights_only=True
]

In [ ]:
#! pip uninstall tensorflow
#! pip install tensorflow==2.3.0

In [18]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
mouthModel1 = tf.keras.models.load_model(r"/content/drive/MyDrive/MouthModel-0.943.h5")
mouthModel1._name = 'mouthModel1'
mouthModel2 = tf.keras.models.load_model(r"/content/drive/MyDrive/MouthModel-0.965.h5")
mouthModel2._name = 'mouthModel2'
mouthModel3 = tf.keras.models.load_model(r"/content/drive/MyDrive/MouthModel-0.960.h5")
mouthModel3._name = 'mouthModel3'

models = [mouthModel1, mouthModel2,mouthModel3]
model_input = Input(shape=(50, 50, 1))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
ensemble_model.save('MouthEnsemble.h5')

In [20]:
mModel1 = tf.keras.models.load_model(r"/content/MouthEnsemble.h5")
mModel1._name = 'ensemble'

In [21]:
(loss,acc) = mModel1.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()

Model Evaluation
Loss: 0.121
Accuracy: 0.967



In [22]:
(loss,acc) = ensemble_model.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()

Model Evaluation
Loss: 0.121
Accuracy: 0.967



In [ ]:
(loss,acc) = eyeModel1.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()

In [ ]:
(loss,acc) = eyeModel2.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()